In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from keras.wrappers.scikit_learn import KerasClassifier
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
import nltk
from nltk.corpus import stopwords

In [2]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
'''import os            ##  This module is for "operating system" interfaces
import sys           ##  This module is for functionality relevant to the python run time

GOOGLE_PATH_AFTER_MYDRIVE = 'NLP_Textcat/spooky_data/train'
GOOGLE_DRIVE_PATH = os.path.join('drive','My Drive', GOOGLE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

# Append the directory path of this notebook to what python easily "sees"
sys.path.append(GOOGLE_DRIVE_PATH)

# Make your current working direct
GOOGLE_DRIVE_PATH'''

'import os            ##  This module is for "operating system" interfaces\nimport sys           ##  This module is for functionality relevant to the python run time\n\nGOOGLE_PATH_AFTER_MYDRIVE = \'NLP_Textcat/spooky_data/train\'\nGOOGLE_DRIVE_PATH = os.path.join(\'drive\',\'My Drive\', GOOGLE_PATH_AFTER_MYDRIVE)\nprint(os.listdir(GOOGLE_DRIVE_PATH))\n\n# Append the directory path of this notebook to what python easily "sees"\nsys.path.append(GOOGLE_DRIVE_PATH)\n\n# Make your current working direct\nGOOGLE_DRIVE_PATH'

In [4]:
import os            ##  This module is for "operating system" interfaces
import sys           ##  This module is for functionality relevant to the python run time
path_to_datafolder = 'C:/Users/mjdom/source/repos/mdst_nlp_2021/data'
print(os.listdir(path_to_datafolder))

['test.csv', 'train.csv']


In [5]:


df = pd.read_csv(path_to_datafolder+ '/train.csv')
df_kaggle = pd.read_csv(path_to_datafolder + '/test.csv')
df_kaggle.head()

#np.hstack((X,df_kaggle['text']))[0]

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [6]:
X = df["text"].copy()
#X = df["text"]

authors = df["author"].copy()

# Label data
y = []
for author in authors:
    if author == "EAP":
        y.append([1, 0, 0])
    if author == "HPL":
        y.append([0, 1, 0])
    if author == "MWS":
        y.append([0, 0, 1])

y = np.array(y)


In [7]:
from tensorflow import keras
from tensorflow.keras import layers

import keras

from keras import backend as K

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)



    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)



class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from keras.wrappers.scikit_learn import KerasClassifier
sequence_length = 100
max_features = 1000000
# Token locations
Vectorizer_transformer = tf.keras.layers.TextVectorization(max_tokens=max_features,output_sequence_length=sequence_length) 
Vectorizer_transformer.adapt(np.hstack((X,df_kaggle['text'])))
vocab = Vectorizer_transformer.get_vocabulary()
vocab_size = len(vocab)


def create_model(embed_dim=32,num_heads = 2,ff_dim = 32,dropout_rate = 0.2):
    
    # create model
    tf.keras.backend.clear_session()
    K.clear_session()
    #gc.collect()



    maxlen = sequence_length

    ## Build embedding and transformer
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim,dropout_rate)

    ## Connect Keras Layers
    inputs = tf.keras.Input(shape=(1,), dtype=tf.string) 
    vec = Vectorizer_transformer(inputs)
    x = embedding_layer(vec)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(3, activation="softmax")(x)

    transformer = keras.Model(inputs=inputs, outputs=outputs)
    transformer.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])
    return transformer

In [12]:
'''import types
import copy
class KerasClassifier(KerasClassifier):
    """ adds sparse matrix handling using batch generator
    """
    
    def fit(self, x, y, **kwargs):
        """ adds sparse matrix handling """
        #if not issparse(x):
        #    return super().fit(x, y, **kwargs)
        
        ############ adapted from KerasClassifier.fit   ######################   
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif not isinstance(self.build_fn, types.FunctionType):
            self.model = self.build_fn(
                **self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
 
        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)
        ### fit => fit_generator
        fit_args = copy.deepcopy(self.filter_sk_params(keras.Model.fit_generator))
        print('gen')
        fit_args.update(kwargs)
        ############################################################
        self.model.fit_generator(
                    self.get_batch(x, y, self.sk_params["batch_size"]),
                                        steps_per_epoch=x.shape[0],
                                        **fit_args)                      
        return self                               
 
    def get_batch(self, x, y=None, batch_size=32):
        """ batch generator to enable sparse input """
        index = np.arange(x.shape[0])
        start = 0
        while True:
            if start == 0 and y is not None:
                np.random.shuffle(index)
            batch = index[start:start+batch_size]
            if y is not None:
                yield x[batch].toarray(), y[batch]
            else:
                yield x[batch].toarray()
            start += batch_size
            if start >= x.shape[0]:
                start = 0
   
    def predict_proba(self, x):
        """ adds sparse matrix handling """
        if not issparse(x):
            return super().predict_proba(x)
            
        preds = self.model.predict_generator(
                    self.get_batch(x, None, self.sk_params["batch_size"]), 
                                               val_samples=x.shape[0])
        return preds'''

'import types\nimport copy\nclass KerasClassifier(KerasClassifier):\n    """ adds sparse matrix handling using batch generator\n    """\n    \n    def fit(self, x, y, **kwargs):\n        """ adds sparse matrix handling """\n        #if not issparse(x):\n        #    return super().fit(x, y, **kwargs)\n        \n        ############ adapted from KerasClassifier.fit   ######################   \n        if self.build_fn is None:\n            self.model = self.__call__(**self.filter_sk_params(self.__call__))\n        elif not isinstance(self.build_fn, types.FunctionType):\n            self.model = self.build_fn(\n                **self.filter_sk_params(self.build_fn.__call__))\n        else:\n            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))\n \n        loss_name = self.model.loss\n        if hasattr(loss_name, \'__name__\'):\n            loss_name = loss_name.__name__\n        if loss_name == \'categorical_crossentropy\' and len(y.shape) != 2:\n            y =

In [17]:
import time
# fix random seed for reproducibility
seed = 15
np.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, batch_size=64, verbose=0)
# define the grid search parameters
embed_dim = [32,64,128,256]
num_heads = [1,2]
ff_dim =  [32,64,128,256]
dropout_rate = [0.0,0.1,0.2,0.3]
epochs = [1,2]

param_grid = dict(embed_dim=embed_dim,num_heads = num_heads,ff_dim = ff_dim,
                  dropout_rate = dropout_rate, epochs=epochs)
#grid = GridSearchCV(estimator=model, scoring = 'neg_log_loss', param_grid=param_grid, n_jobs=1, cv=3, verbose=3)
grid = RandomizedSearchCV(model, param_grid, n_iter=2,scoring = 'neg_log_loss', n_jobs=1, cv=5, verbose=3)

C:\Users\mjdom\AppData\Local\Temp/ipykernel_17948/1202741136.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, batch_size=64, verbose=0)


In [21]:

start_time = time.time()
grid_result = grid.fit(X, y)
stop_time = time.time()

# summarize results
print('time search took:', stop_time - start_time)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END dropout_rate=0.2, embed_dim=32, epochs=2, ff_dim=128, num_heads=1;, score=nan total time=   0.0s
[CV 2/5] END dropout_rate=0.2, embed_dim=32, epochs=2, ff_dim=128, num_heads=1;, score=nan total time=   0.0s
[CV 3/5] END dropout_rate=0.2, embed_dim=32, epochs=2, ff_dim=128, num_heads=1;, score=nan total time=   0.0s
[CV 4/5] END dropout_rate=0.2, embed_dim=32, epochs=2, ff_dim=128, num_heads=1;, score=nan total time=   0.0s
[CV 5/5] END dropout_rate=0.2, embed_dim=32, epochs=2, ff_dim=128, num_heads=1;, score=nan total time=   0.0s
[CV 1/5] END dropout_rate=0.2, embed_dim=32, epochs=1, ff_dim=128, num_heads=2;, score=nan total time=   0.0s
[CV 2/5] END dropout_rate=0.2, embed_dim=32, epochs=1, ff_dim=128, num_heads=2;, score=nan total time=   0.0s
[CV 3/5] END dropout_rate=0.2, embed_dim=32, epochs=1, ff_dim=128, num_heads=2;, score=nan total time=   0.0s
[CV 4/5] END dropout_rate=0.2, embed_dim=32, epochs=1, ff_di

C:\Users\mjdom\anaconda3\envs\pipgpu\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mjdom\anaconda3\envs\pipgpu\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mjdom\anaconda3\envs\pipgpu\lib\site-packages\keras\wrappers\scikit_learn.py", line 232, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\mjdom\anaconda3\envs\pipgpu\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit


AssertionError: Do not use tf.reset_default_graph() to clear nested graphs. If you need a cleared graph, exit the nesting and create a new graph.

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
d=pd.DataFrame(params)
d['Mean']=means
d['Std. Dev']=stds

In [ ]:
param_ = ["num_heads","ff_dim","epochs","embed_dim","dropout_rate"]
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2,3,figsize=(14,8), squeeze=False)
ax = ax.ravel()
for i in range(5):
    ax[i].set_title('Distribution of mean accuracy with {}'.format(param_[i]))
    sns.violinplot(x=param_[i],y='Mean',data=d,ax=ax[i])
fig.tight_layout(pad=1.5)

In [ ]:
d.sort_values(by='Mean',ascending = False)


In [ ]:
nltk.download('stopwords')
stop = stopwords.words('english')
len(stop)
s_upper = []
for s in stop:
    s_upper.append(s[0].upper()+s[1:])

all_stop = stop + s_upper

len(all_stop)

In [ ]:
X_stop = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (all_stop)]))


In [ ]:
import time

# fix random seed for reproducibility
seed = 15
np.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, batch_size=64, verbose=0)
# define the grid search parameters
embed_dim = [32,64,128,256]
num_heads = [1,2]
ff_dim =  [32,64,128,256]
dropout_rate = [0.0,0.1,0.2,0.3]
epochs = [1,2]

param_grid = dict(embed_dim=embed_dim,num_heads = num_heads,ff_dim = ff_dim,
                  dropout_rate = dropout_rate, epochs=epochs)
#grid = GridSearchCV(estimator=model, scoring = 'neg_log_loss', param_grid=param_grid, n_jobs=1, cv=3, verbose=3)
grid = RandomizedSearchCV(model, param_grid, n_iter=100,scoring = 'neg_log_loss', n_jobs=1, cv=5, verbose=3)

In [ ]:
start_time = time.time()
grid_result = grid.fit(X_stop, y)
stop_time = time.time()

# summarize results
print('time search took:', stop_time - start_time)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
d=pd.DataFrame(params)
d['Mean']=means
d['Std. Dev']=stds

In [ ]:
param_ = ["num_heads","ff_dim","epochs","embed_dim","dropout_rate"]
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2,3,figsize=(14,8), squeeze=False)
ax = ax.ravel()
for i in range(5):
    ax[i].set_title('Distribution of mean accuracy with {}'.format(param_[i]))
    sns.violinplot(x=param_[i],y='Mean',data=d,ax=ax[i])
fig.tight_layout(pad=1.5)

In [ ]:
d.sort_values(by='Mean',ascending = False)


In [ ]:
import multiprocessing

def run_inference_or_training(X, y):
    grid_result = grid.fit(X, y)

#if __name__ == '__main__':
p = multiprocessing.Process(
    target=run_inference_or_training,
    args=(X, y, )
)
p.start()
p.join()

In [20]:
tf.config.experimental.get_memory_info('GPU:0')

{'current': 2304, 'peak': 232192}

In [20]:
keras.backend.clear_session()

In [18]:
with tf.Graph().as_default():


NameError: name 'main' is not defined

In [ ]:
import gc

In [ ]:
709750016/777168128

In [ ]:
del grid

In [18]:
import multiprocessing
import proc
def run_inference_or_training2(X, y):
    print(X,y)

#if __name__ == '__main__':
p = multiprocessing.Process(
    target=proc.test2,
    args=(2, 3, )
)
p.start()
print(p)
p.join()

<Process name='Process-4' pid=4276 parent=17948 started>


In [10]:
from multiprocessing import Pool
if __name__ ==  '__main__': 
    num_processors = 1
    p=Pool(processes = num_processors)
    output = p.map(proc.test2,[(1),(2)])
    print(output)

[1, 2]


In [19]:
p = multiprocessing.Process(
    target=proc.run_grid,
    args=(X, y, grid)
)
p.start()
p.join()